In [1]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen
plotly();

### Constantes

In [2]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference

### Modele pour une microlentille

In [3]:
bbox = BoundingBox(xmin=1, ymin=1, xmax=15, ymax=50); #size of bbox
@show a0= 1. .- 0.2 .* randn(Float64,length(λlaser));
@show fwhm0= 5. .-  2. .* rand(Float64,length(λlaser));
laser =  LaserModel(λlaser,a0,fwhm0);
lmod = LensletModel(λ0,laser.nλ-1,bbox);

C0 = zeros(2,laser.nλ)
@show C0[1:2,1:2] = [[ 6.2 10]; [25 80]]

#affichage de l'image
UpdateDispModel(lmod.dmodel,C0)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)

a0 = 1.0 .- 0.2 .* randn(Float64, length(λlaser)) = [1.0561309045643972, 1.152710401417176, 1.151099317026695]
fwhm0 = 5.0 .- 2.0 .* rand(Float64, length(λlaser)) = [3.043278514085292, 3.068569383978945, 3.4963421255308016]
C0[1:2, 1:2] = [[6.2 10]; [25 80]] = [6.2 10.0; 25.0 80.0]


<!DOCTYPE html>
 
 
 Plots.jl

In [4]:
data = limage .+ 0.05 .*randn(Float64,size(limage)) ; #ajout de bruit 
heatmap(data)

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
#Tableau
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

@show cost = likelihood(a0,fwhm0,C0)
∇cost = gradient(likelihood,a0,fwhm0,C0);

cost = likelihood(a0, fwhm0, C0) = 1.839333632368493


In [6]:
#Avec bruit
@show cost = likelihood(a0,fwhm0.+ 2.,C0)
∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,C0);

cost = likelihood(a0, fwhm0 .+ 2.0, C0) = 17.738773626713915


In [7]:
#Vecteur: optimisation plus rapide car les trois coefficients a traiter sont donnes en meme temps
function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:(laser.nλ)],x[(laser.nλ+1):(2*laser.nλ)],reshape(x[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

x0 = vcat([a0[:],fwhm0[:],C0[:] ]...) #concatenation de a0, fwhm0 et C0

@show cost = likelihood(x0)
∇cost = gradient(likelihood,x0);

cost = likelihood(x0) = 1.839333632368493


### Optimisation

In [8]:
#Parametres d'initialisation
ainit = a0  .+ (rand(Float64,laser.nλ) .- 0.5);
fwhminit = fwhm0  .+ (rand(Float64,laser.nλ) .- 0.5);
cinit = zeros(2,laser.nλ)
cinit[1:2,1:2] = [[ 6.2 0]; [25 60]]

UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);

xinit = vcat([ainit[:],fwhminit[:],cinit[:]]...)

∇cost = gradient(likelihood,xinit )

([4.585741032730125, -4.7314205851284346, 6.31408128731677, 0.10395220271750533, -1.6636754722390836, 0.3455595317625701, 0.08258556381460425, 0.012639050732197621, -0.32111744996361724, -0.6227209359540125, 0.011305901789280946, 0.020444224365821294],)

In [9]:
#Parametres d'optimisation
xopt = vmlmb(likelihood, xinit; verb=50, ftol=(0.0,0),gtol = (0.0,0));

(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));


# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.0968060151444586E+01  9.31E+00  0.00E+00
    50     53      0    1.8373272132342164E+00  2.94E-01  1.00E+00
   100    104      0    1.8027711741620462E+00  2.17E-02  1.00E+00
   150    157      0    1.8027018872127034E+00  1.03E-04  1.00E+00
   200    209      0    1.8027017013228368E+00  1.80E-04  1.00E+00


### Comparaison

In [10]:
@show(ainit,aopt,a0);

@show(fwhminit,fwhmopt,fwhm0);

@show(cinit,copt,C0);

heatmap( data .- LensletLaserImage(lmod,laser)) #Residus

ainit = [0.6850303529100545, 0.6912655294445416, 0.9109697035079583]
aopt = [1.0297542706492195, 1.1341262000186536, 1.1894202502639288]
a0 = [1.0561309045643972, 1.152710401417176, 1.151099317026695]
fwhminit = [3.2273763932294592, 2.6525969218837857, 3.0415089372656503]
fwhmopt = [3.178884102908544, 3.076286963233277, 3.460936020233228]
fwhm0 = [3.043278514085292, 3.068569383978945, 3.4963421255308016]
cinit = [6.2 0.0 0.0; 25.0 60.0 0.0]
copt = [6.167308613116124 9.801094424382724 -0.45938260019189386; 24.958919474842904 79.85987547044294 1.9442835603817556]
C0 = [6.2 10.0 0.0; 25.0 80.0 0.0]


<!DOCTYPE html>
 
 
 Plots.jl

In [11]:
#Image finale
heatmap(data)

<!DOCTYPE html>
 
 
 Plots.jl